In [26]:
deployment = "gpt4"
model = "gpt-4"

In [27]:
def get_current_cluster_state(cluster_name):
    return  """ERROR: Failed to pull image "chaocai/docker/dsp:latest"""

In [34]:
import openai
import json

openai.api_version = "2023-07-01-preview"

funcs = {"get_current_cluster_state": get_current_cluster_state}


def run(input):
    msg=[{"role":"user","content":input}]
    ret = run_conversation(msg)
    return ret["content"] 

def run_conversation(msg):
    response = openai.ChatCompletion.create(
        engine=deployment,
        model=model,
        messages=msg,
        functions=[
            {
                "name": "get_current_cluster_state",
                "description": "Get the current state in a given cluster",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "cluster_name": {
                            "type": "string",
                            "description": "the name of the cluster",
                        },
                        
                    },
                    "required": ["cluster_name"],
                },
            }
        ],
        function_call="auto",
    )

    message =  response["choices"][0]["message"]
    
    # 如果不需要调用function，则直接返回结果
    if not message.get("function_call"):
        return message
    
    # 从
    function_name = message["function_call"]["name"]
    function_args = json.loads(message["function_call"]["arguments"])
    print(function_args)
    res = funcs[function_name](function_args["cluster_name"])
    message["content"]=None
    msg.append(message)
    msg.append({
                "role": "function",
                "name": function_name,
                "content": res,
            })
    return run_conversation(msg)
   

In [33]:
#run("Hello")
print(run("What's wrong with the cluster 'DSP'? And if there's an error, give me some suggestion. "))

{'cluster_name': 'DSP'}
The error indicates the system is unable to pull the latest docker image for the 'DSP' cluster. 

Here are a few suggestions to troubleshoot this issue:

1. Check the Docker Hub repository to ensure the image "chaocai/docker/dsp:latest" exists and is accessible.

2. Examine your Docker configurations for any issues and make sure you are logged into Docker Hub. 

3. Verify network connectivity. The server needs to communicate with Docker Hub to pull the image.

4. Ensure you have enough storage space available. The image may not be pulled if there is insufficient storage. 

5. If the problem persists, it could be due to a Docker daemon issue. Try restarting it. 

Remember to check the logs for any additional error details or warnings. This might provide more context on the issue.
